In [ ]:
# import necessary libraries

import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
#import glove
#from glove import Corpus

import collections
import gc

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D

from keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import medical entities dataset
med7_ner_data = pd.read_pickle("data/new_ner_word_dict.pkl")

#### Exploring the medical entities dataset

In [ ]:
# Get the unique entities of med7 model

unique_categories = set()

for values in med7_ner_data.values():
    for item in values:
        category = item[1]
        unique_categories.add(category)

print(f"Unique words (categories) in the values: {unique_categories}")

Unique words (categories) in the values:
{'FREQUENCY', 'DURATION', 'STRENGTH', 'FORM', 'DRUG', 'DOSAGE', 'ROUTE'}


In [ ]:
# Print the total counts for each med7 entity

counts = {category: 0 for category in unique_categories}

for values in med7_ner_data.values():
    for item in values:
        category = item[1]
        counts[category] += 1

print("Total count for each category:")
for category, count in counts.items():
    print(f"Total counts of {category}: {count}")

Total count for each category:
Total counts of FREQUENCY: 120210
Total counts of DURATION: 9688
Total counts of STRENGTH: 219455
Total counts of FORM: 67075
Total counts of DRUG: 979999
Total counts of DOSAGE: 250311
Total counts of ROUTE: 263403


In [ ]:
# Print the unique counts for each med7 entity and an example

unique_values_per_category = {category: set() for category in unique_categories}

for values in med7_ner_data.values():
    for item in values:
        value, category = item
        unique_values_per_category[category].add(value)

print("Unique count for each category:")
for category, unique_values in unique_values_per_category.items():
    print(f"Unique counts of {category}: {len(unique_values)}")
    print(f"{category} example: {list(unique_values)[12]}")

Unique count for each category:
Unique counts of FREQUENCY: 5444
FREQUENCY example: tue 11:18 pm
Unique counts of DURATION: 1678
DURATION example: 10.3 -
Unique counts of STRENGTH: 10067
STRENGTH example: 10-20 mg
Unique counts of FORM: 885
FORM example: xrays
Unique counts of DRUG: 17502
DRUG example: madaz
Unique counts of DOSAGE: 4191
DOSAGE example: 1500cc
Unique counts of ROUTE: 1149
ROUTE example: oozing


In [ ]:
# Check if running code on CPU or GPU
import tensorflow as tf

# Enable device logging
#tf.debugging.set_log_device_placement(True)
tf.debugging.set_log_device_placement(False)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


#### Timeseries Model

In [ ]:
# Helper functions for training the time series model

def reset_keras(model):
    """reset keras Session"""
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted

def make_prediction_timeseries(model, test_data):
    """make model predictions"""
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_timeseries(predictions, probs, ground_truth, model_name,
                problem_type, iteration, hidden_unit_size, type_of_ner):
    """save metrics from model predictions"""
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)

    result_dict = {}
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1

    file_name = str(hidden_unit_size)+"-"+model_name+"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+".p"

    result_path = "results/"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)

In [ ]:
# Define the time series model

def timeseries_model(layer_name, number_of_unit):
    """define time series model specifications"""
    K.clear_session()

    sequence_input = Input(shape=(24,104),  name = "timeseries_input")

    if layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)
    else:
        x = GRU(number_of_unit)(sequence_input)

    #logits_regularizer = tf.contrib.layers.l2_regularizer(scale=0.01)
    logits_regularizer = keras.regularizers.l2(0.01)
    sigmoid_pred = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.GlorotUniform(),
                  kernel_regularizer=logits_regularizer)(x)

    model = Model(inputs=sequence_input, outputs=sigmoid_pred)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [ ]:
# import dataset
type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")

In [ ]:
# Train and test the time series model
epoch_num = 100
model_patience = 3
monitor_criteria = 'val_loss'
batch_size = 128

unit_sizes = [128, 256]
#unit_sizes = [256]
iter_num = 11
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']
layers = ["LSTM", "GRU"]
#layers = ["GRU"]
for each_layer in layers:
    print("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print("Hidden unit: ", each_unit_size)
        for iteration in range(1, iter_num):
            print("Iteration number: ", iteration)
            print("=============================")

            for each_problem in target_problems:
                print ("Problem type: ", each_problem)
                print ("__________________")


                early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                best_model_name = str(each_layer)+"-"+str(each_unit_size)+"-"+str(each_problem)+"-"+"best_model.keras"
                checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                    save_best_only=True, mode='min')


                callbacks = [early_stopping_monitor, checkpoint]

                model = timeseries_model(each_layer, each_unit_size)
                model.fit(x_train_lstm, y_train[each_problem], epochs=epoch_num, verbose=1,
                          validation_data=(x_dev_lstm, y_dev[each_problem]), callbacks=callbacks, batch_size= batch_size)

                model.load_weights(best_model_name)

                probs, predictions = make_prediction_timeseries(model, x_test_lstm)
                save_scores_timeseries(predictions, probs, y_test[each_problem].values,str(each_layer),
                                       each_problem, iteration, each_unit_size,type_of_ner)
                reset_keras(model)
                #del model
                clear_session()
                gc.collect()

Layer:  LSTM
Hidden unit:  128
Iteration number:  1
Problem type:  mort_hosp
__________________
Epoch 1/100
121/122 [============================>.] - ETA: 0s - loss: 0.2977 - acc: 0.8926
Epoch 1: val_loss improved from inf to 0.25256, saving model to LSTM-128-mort_hosp-best_model.keras
122/122 [==============================] - 13s 79ms/step - loss: 0.2979 - acc: 0.8927 - val_loss: 0.2526 - val_acc: 0.9129
Epoch 2/100
121/122 [============================>.] - ETA: 0s - loss: 0.2356 - acc: 0.9180
Epoch 2: val_loss improved from 0.25256 to 0.24693, saving model to LSTM-128-mort_hosp-best_model.keras
122/122 [==============================] - 9s 73ms/step - loss: 0.2355 - acc: 0.9180 - val_loss: 0.2469 - val_acc: 0.9152
Epoch 3/100
121/122 [============================>.] - ETA: 0s - loss: 0.2131 - acc: 0.9274
Epoch 3: val_loss improved from 0.24693 to 0.24652, saving model to LSTM-128-mort_hosp-best_model.keras
122/122 [==============================] - 9s 72ms/step - loss: 0.2136 - ac

#### Show average over iterations for each metric

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd

# Define categories and metrics
categories = ["128-GRU", "256-GRU", "128-LSTM", "256-LSTM"]
metrics = {"auc":"AUROC", "auprc":"AUPRC", "acc":"Accuracy", "F1":"F1"}
tasks = ["mort_hosp", "mort_icu", "los_3", "los_7"]

# Initialize dictionaries to store results
results = {category: {task: {metric_name: [] for metric, metric_name in metrics.items()} for task in tasks} for category in categories}

# Directory where pickle files are stored
directory = "results/"

# Loop through each file
for filename in os.listdir(directory):
    if filename.endswith(".p"):
        parts = filename.split("-")
        category = parts[0] + "-" + parts[1]
        task = parts[2]
        if category in categories and task in tasks:
            result_dict = pd.read_pickle(os.path.join(directory, filename))
            for metric, metric_name in metrics.items():
                results[category][task][metric_name].append(result_dict[metric])

# Calculate average and standard deviation
"""
for category in categories:
    print(f"Category: {category}")
    for task in tasks:
        print(f"Task: {task}")
        for metric in metrics:
            values = results[category][task][metric]
            avg = np.mean(values)
            std = np.std(values)
            print(f"  {metric}: Avg = {avg}, Std = {std}")
"""

for category in categories:
    print(f"Category: {category}")
    df_data = {task: {} for task in tasks}
    for task in tasks:
        task_data = {}
        for metric, metric_name in metrics.items():
            values = results[category][task][metric_name]
            mean = np.mean(values)
            std = np.std(values)
            task_data[metric_name] = f"{mean:.4f} \u00B1 {std:.4f}"
        df_data[task] = task_data
    df = pd.DataFrame(df_data).transpose()
    print(df)
    print()


Category: 128-GRU
                     AUROC            AUPRC         Accuracy               F1
mort_hosp  0.8790 ± 0.0020  0.5661 ± 0.0065  0.9141 ± 0.0018  0.4421 ± 0.0172
mort_icu   0.8894 ± 0.0031  0.5171 ± 0.0086  0.9407 ± 0.0014  0.4286 ± 0.0225
los_3      0.6960 ± 0.0037  0.6363 ± 0.0044  0.6607 ± 0.0044  0.5591 ± 0.0077
los_7      0.7376 ± 0.0052  0.2117 ± 0.0075  0.9183 ± 0.0011  0.0493 ± 0.0164

Category: 256-GRU
                     AUROC            AUPRC         Accuracy               F1
mort_hosp  0.8766 ± 0.0019  0.5561 ± 0.0068  0.9130 ± 0.0017  0.4363 ± 0.0081
mort_icu   0.8891 ± 0.0037  0.5212 ± 0.0078  0.9412 ± 0.0013  0.4429 ± 0.0190
los_3      0.6929 ± 0.0033  0.6369 ± 0.0043  0.6573 ± 0.0035  0.5469 ± 0.0114
los_7      0.7374 ± 0.0077  0.2132 ± 0.0051  0.9184 ± 0.0008  0.0427 ± 0.0175

Category: 128-LSTM
                     AUROC            AUPRC         Accuracy               F1
mort_hosp  0.8756 ± 0.0035  0.5497 ± 0.0091  0.9119 ± 0.0022  0.4361 ± 0.0229
mort_ic